In [4]:
# Importing the pandas library.
import pandas as pd
import matplotlib as plt
import copy

# Using pandas' read_csv function to read the data.
data= pd.read_csv('AKBNK.E.csv')
#adding column names to the data frame
column_names=["network_time", "bist_time", "msg_type", "asset_name", "side", "price", "que_loc", "qty", "order_id"]
data.columns=column_names
#removing unnecessary orders from the data frame(only the add execute and dlete orders should be there) 
data = data[data["msg_type"].isin(["A", "E", "D"])]
# Reset the indexes to start from 0
data.reset_index(drop=True, inplace=True)

#converting unix time to date-time objects
data['network_time'] = pd.to_datetime(data['network_time'], unit='ns')
data['bist_time'] = pd.to_datetime(data['bist_time'], unit='ns')

data.head()

,network_time,bist_time,msg_type,asset_name,side,price,que_loc,qty,order_id
0,2022-09-21 06:55:26.244327990,2022-09-21 06:55:26.145096528,A,AKBNK.E,B,11.81,1,6000,7621969089428617284
1,2022-09-21 06:55:26.244327990,2022-09-21 06:55:26.145096528,A,AKBNK.E,B,11.81,2,100,7621969089428723717
2,2022-09-21 06:55:26.244327990,2022-09-21 06:55:26.145096528,A,AKBNK.E,B,11.81,3,10,7621969089428724890
3,2022-09-21 06:55:26.244327990,2022-09-21 06:55:26.145096528,A,AKBNK.E,B,11.81,4,5000,7621969089428940862
4,2022-09-21 06:55:26.244327990,2022-09-21 06:55:26.145096528,A,AKBNK.E,B,11.80,5,1,7621969089428616329


In [5]:
#reading the sample lob
lob_sample=pd.read_csv("sample_lob.txt")
#looping through the data to construct the order book for each network time stemp
lob_sample.head()

,Date,Asset,bid3qty,bid3px,bid2qty,bid2px,bid1qty,bid1px,ask1px,ask1qty,ask2px,ask2qty,ask3px,ask3qty,Mold Package
0,2022-09-21 09:55:26.244327936,AKBNK,10.0,11.78,51822.0,11.8,11110.0,11.81,0.0,0.0,0.0,0.0,0.0,0.0,A-B-11.81-6000-7621969089428617284;A-B-11.81-1...
1,2022-09-21 09:55:26.244354816,AKBNK,10.0,11.78,51822.0,11.8,11110.0,11.81,0.0,0.0,0.0,0.0,0.0,0.0,A-B-11.76-1000-7621969089428889683;A-B-11.75-5...
2,2022-09-21 09:55:26.244382464,AKBNK,10.0,11.78,51822.0,11.8,11110.0,11.81,0.0,0.0,0.0,0.0,0.0,0.0,A-B-11.65-2-7621969089428687845;A-B-11.65-5000...
3,2022-09-21 09:55:26.244411648,AKBNK,10.0,11.78,51822.0,11.8,11110.0,11.81,0.0,0.0,0.0,0.0,0.0,0.0,A-B-11.55-30-7621969089428656552;A-B-11.55-60-...
4,2022-09-21 09:55:26.244439808,AKBNK,10.0,11.78,51822.0,11.8,11110.0,11.81,0.0,0.0,0.0,0.0,0.0,0.0,A-B-11.5-100-7621969089428683810;A-B-11.5-100-...


In [6]:
default_values = {
        'Date': pd.to_datetime(0, unit='s'),
        'Asset': 'AKBNK',  
        'bid3qty': 0,
        'bid3px': 0,
        'bid2qty': 0,
        'bid2px': 0,
        'bid1qty': 0,
        'bid1px': 0,
        'ask1px': 0,
        'ask1qty': 0,
        'ask2px': 0,
        'ask2qty': 0,
        'ask3px': 0,
        'ask3qty': 0,
        'id3qty': 0,
        'Mold Package' : str()
    }

lob=pd.DataFrame([default_values])

In [17]:
import timeit
start_time=timeit.default_timer()

# looping through the data to construct the order book for each network time stemp
for data_index, row in data.iterrows():
    last_index=lob.index[-1]
    # if it is an add order
    if row['msg_type'] == 'A':
        # if it is a buy
        if row['side'] == 'B':
            # if its price is better than the current bids
            if row['price'] >= lob.loc[last_index, 'bid1px']:
                #creating a string representing the order and adding it to the Mold Package
                #mp_str=last_row['Mold Package']+';'+'A-B-'+row['price']+'-'+row['qty']+row['order_id'] 
                # if the times are equal, update the existing row
                if lob.loc[last_index,'Date'] == row['network_time'] or lob.loc[last_index,'Date']==pd.to_datetime(0,unit='s'):
                    #assigning the price and the quantity
                    lob.loc[last_index,'bid2px']=lob.loc[last_index,'bid1px']
                    lob.loc[last_index,'bid3px']=lob.loc[last_index,'bid2px']
                    lob.loc[last_index,'bid2qty']=lob.loc[last_index,'bid1qty']
                    lob.loc[last_index,'bid3qty']=lob.loc[last_index,'bid2qty']
                    lob.loc[last_index, 'bid1qty'] = row['qty']
                    lob.loc[last_index, 'bid1px'] = row['price']
                    #assigning the updated mold package str
                    #last_row['Mold Package']=mp_str
                # delta time is positive so a new row should be added to the lob df
                else:
                    # add a new row to the lob with the updated 'bid3qty' 'bid3px' values. It's date should be row['network_time']
                    new_dict=copy.deepcopy(default_values)
                    new_dict['Asset']=row['asset_name']
                    new_dict['bid3qty']=row['qty']+lob.loc[last_index,'bid3qty']
                    new_dict['bid3px']=row['price']
                    #adding other bids to the new dict becaus ethey are unchanged
                    new_dict['bid2px']=lob.loc[last_index, 'bid2px'] 
                    new_dict['bid3px']=lob.loc[last_index, 'bid3px']
                    new_dict['bid2qty']=lob.loc[last_index, 'bid2qty']
                    new_dict['bid3qty']=lob.loc[last_index, 'bid3qty']
                    new_row = pd.DataFrame([new_dict])
                    lob = pd.concat([lob, new_row], ignore_index=True)
                                
                lob.loc[last_index, 'Date']=row['network_time']
                continue
            
            if row['price'] >= lob.loc[last_index, 'bid2px']:
                # if the times are equal, update the existing row
                if last_row['Date'] == row['network_time'] or lob.loc[last_index,'Date']==pd.to_datetime(0,unit='s'):
                    #assigning the price and the quantity
                    # Modifying the values directly in lob
                    lob.loc[last_index,'bid2px']=lob.loc[last_index,'bid1px']
                    lob.loc[last_index,'bid3px']=lob.loc[last_index,'bid2px']
                    lob.loc[last_index, 'bid2qty'] = row['qty']
                    lob.loc[last_index, 'bid2px'] = row['price']
                    #assigning the updated mold package str
                    #last_row['Mold Package']=mp_str
                # delta time is positive so a new row should be added to the lob df
                else:
                    # add a new row to the lob with the updated 'bid3qty' 'bid3px' values. It's date should be row['network_time']
                    new_dict=copy.deepcopy(default_values)
                    new_dict['Asset']=row['asset_name']
                    new_dict['bid2qty']=row['qty']+lob.loc[last_index,'bid2qty']
                    new_dict['bid2px']=row['price']
                    #adding other bids to the new dict becaus ethey are unchanged
                    new_dict['bid1px']=lob.loc[last_index, 'bid1px'] 
                    new_dict['bid3px']=lob.loc[last_index, 'bid3px']
                    new_dict['bid1qty']=lob.loc[last_index, 'bid1qty']
                    new_dict['bid3qty']=lob.loc[last_index, 'bid3qty']
                    new_row = pd.DataFrame([new_dict])
                    lob = pd.concat([lob, new_row], ignore_index=True)
                                
                lob.loc[last_index, 'Date']=row['network_time']
                continue
            if row['price'] >= lob.loc[last_index, 'bid3px']:
                # if the times are equal, update the existing row
                if last_row['Date'] == row['network_time'] or lob.loc[last_index,'Date']==pd.to_datetime(0,unit='s'):
                    #assigning the price and the quantity
                    # Modifying the values directly in lob
                    lob.loc[last_index,'bid2px']=lob.loc[last_index,'bid1px']
                    lob.loc[last_index,'bid3px']=lob.loc[last_index,'bid2px']
                    lob.loc[last_index, 'bid3qty'] = row['qty']
                    lob.loc[last_index, 'bid3px'] = row['price']
                    #assigning the updated mold package str
                    #last_row['Mold Package']=mp_str
                # delta time is positive so a new row should be added to the lob df
                else:
                    # add a new row to the lob with the updated 'bid3qty' 'bid3px' values. It's date should be row['network_time']
                    new_dict=copy.deepcopy(default_values)
                    new_dict['Asset']=row['asset_name']
                    new_dict['bid3qty']=row['qty']+lob.loc[last_index,'bid3qty']
                    new_dict['bid3px']=row['price']
                    #adding other bids to the new dict becaus ethey are unchanged
                    new_dict['bid1px']=lob.loc[last_index, 'bid1px'] 
                    new_dict['bid2px']=lob.loc[last_index, 'bid2px']
                    new_dict['bid1qty']=lob.loc[last_index, 'bid1qty']
                    new_dict['bid2qty']=lob.loc[last_index, 'bid2qty']
                    new_row = pd.DataFrame([new_dict])
                    lob = pd.concat([lob, new_row], ignore_index=True)
                                
                lob.loc[last_index, 'Date']=row['network_time']
                continue
                
        # if it is a sell
        elif row['side'] == 'S':
            # if its price is better than the current asks
            pass
    
    # if the mesage type is delete
    elif row['msg_type'] == 'D':
        # Action for delete message type
        pass
    
    # if the mesage type is execution
    elif row['msg_type'] == 'E':
        # Action for execution message type
        pass

end_time=timeit.default_timer()
elapsed_time = end_time - start_time
lob.to_csv("my_lob.txt",sep=",")
print(f"Elapsed time: {elapsed_time} seconds")
lob.head()

Elapsed time: 13.64191870600007 seconds


,Date,Asset,bid3qty,bid3px,bid2qty,bid2px,bid1qty,bid1px,ask1px,ask1qty,ask2px,ask2qty,ask3px,ask3qty,id3qty,Mold Package
0,2022-09-21 09:07:26.738375598,AKBNK,5543,12.05,800,12.06,16914,12.07,0,0,0,0,0,0,0,
1,2022-09-21 09:07:26.739607714,AKBNK.E,11086,12.05,0,0.00,11195,12.05,0,0,0,0,0,0,0,
2,2022-09-21 09:07:26.740612873,AKBNK.E,0,0.00,1380,11.97,100,12.04,0,0,0,0,0,0,0,
3,2022-09-21 09:07:26.744387889,AKBNK.E,13850,12.05,0,0.00,7306,12.05,0,0,0,0,0,0,0,
4,2022-09-21 09:07:27.178427695,AKBNK.E,0,0.00,4000,12.03,7702,12.04,0,0,0,0,0,0,0,
